In [33]:
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from keras import applications
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

datasetpath = r'C:\Users\SurfacePro4\Desktop\dataset'
batch_size = 32
img_size = 128
vggmodel = applications.VGG16(include_top=False, weights='imagenet')

vgg_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split = 0.2)

In [34]:
train_generator = vgg_data_gen.flow_from_directory(
        r'C:\Users\SurfacePro4\Desktop\dataset\training',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=False)  

val_generator = vgg_data_gen.flow_from_directory(
        r'C:\Users\SurfacePro4\Desktop\dataset\validation',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 2197 images belonging to 4 classes.
Found 612 images belonging to 4 classes.


In [ ]:
#extract features from train_generator (take quite long time)
X_train = vggmodel.predict_generator(train_generator, len(train_generator))

#save X_train (so we don't need to extract again)
filename = 'cnn_features_training(i128b32)'
with open(filename, 'wb') as f:
          np.save(f, X_train)

In [ ]:
#extract features from val set
X_val = vggmodel.predict_generator(val_generator, len(val_generator))
filename = 'cnn_features_validation(i128b32)'
with open(filename, 'wb') as f:
          np.save(f, X_val)

**If you already have cnn_features_training/valid files, you can skip extraction cells (2 cells above)

In [31]:
from keras.utils.np_utils import to_categorical
num_classes = len(train_generator.class_indices)
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(val_generator.filenames)

with open('cnn_features_training(i64b32)', 'rb') as f:
    X_train = np.load(f)
Y_train = train_generator.classes
Y_train = to_categorical(Y_train, num_classes=num_classes)

with open('cnn_features_validation(i64b32)', 'rb') as f:
    X_val = np.load(f)
Y_val = val_generator.classes
Y_val = to_categorical(Y_val, num_classes=num_classes)

#datagen for top model (our model)
datagen_top = ImageDataGenerator(rescale=1. / 255)

#top model
model = Sequential()
model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=8)

History = model.fit(X_train, 
                    Y_train,
                    epochs=50,
                    batch_size=batch_size,
                    validation_data=(X_val, Y_val),
                    callbacks=[early_stopping]
                    )

Train on 2197 samples, validate on 612 samples
Epoch 1/50
2197/2197 [==============================] - 2s 888us/step - loss: 7.3276 - acc: 0.5089 - val_loss: 6.2845 - val_acc: 0.5850
Epoch 2/50
2197/2197 [==============================] - 1s 593us/step - loss: 6.2732 - acc: 0.5922 - val_loss: 5.8929 - val_acc: 0.6209
Epoch 3/50
2197/2197 [==============================] - 1s 509us/step - loss: 4.6961 - acc: 0.6796 - val_loss: 3.3377 - val_acc: 0.7614
Epoch 4/50
2197/2197 [==============================] - 1s 509us/step - loss: 3.6035 - acc: 0.7396 - val_loss: 3.2930 - val_acc: 0.7712
Epoch 5/50
2197/2197 [==============================] - 1s 555us/step - loss: 3.0652 - acc: 0.7856 - val_loss: 3.2523 - val_acc: 0.7778
Epoch 6/50
2197/2197 [==============================] - 1s 522us/step - loss: 2.6909 - acc: 0.8079 - val_loss: 3.2114 - val_acc: 0.7810
Epoch 7/50
2197/2197 [==============================] - 1s 561us/step - loss: 2.7246 - acc: 0.8084 - val_loss: 3.2586 - val_acc: 0.7696
E

In [ ]:
import matplotlib.pyplot as plt
def draw_graph(History, arg): #arg = acc|loss
    plt.tight_layout()
    plt.plot(History.history[arg])
    val_arg = 'val_' + arg
    plt.plot(History.history[val_arg])
    plt.title('Model '+ arg)
    plt.ylabel(arg)
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'])
    plt.show()

In [ ]:
plt.plot(History.history['acc'])
plt.plot(History.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
#test on test set
test_generator = vgg_data_gen.flow_from_directory(
        r'C:\Users\SurfacePro4\Desktop\dataset\testing',
        target_size=(img_size, img_size),
#         batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
cnn_features = model.predict_generator(val_generator, len(val_generator))

In [ ]:
from sklearn.metrics import confusion_matrix

def create_cm_for_test(testdatapath = r'C:\Users\SurfacePro4\Desktop\dataset\testing', img_size = 128):
    datagen = ImageDataGenerator(rescale = 1./255)
    test_generator = datagen.flow_from_directory(testdatapath,
                                            target_size = (img_size, img_size),
                                            class_mode = 'categorical',
                                            shuffle = False
                                            )
    Y_pred = model.predict_generator(test_generator, len(test_generator))
    y_pred = np.argmax(Y_pred, axis=1)
    cm = confusion_matrix(test_generator.classes, y_pred)
    plot_confusion_matrix(cm, test_generator.class_indices)
    
create_cm_for_test()